In [2]:
from __future__ import division

import tweepy # Helps in getting tweets 
from textblob import TextBlob # For getting tweet's subjectivity and polarity
from twython import Twython # Helps in getting tweets 
from wordcloud import WordCloud # Helps in forming a word cloud
import pandas as pd # Making and Manipulating a dataframe
import numpy as np # Helps in various vector and matrix calculations
import re # The Regex library
import matplotlib.pyplot as plt # For plotting various visualizations
from functools import reduce # An important method which will help in joining data frames
plt.style.use('fivethirtyeight')
import os # The os library to get current path and various other things
from PIL import Image # To get a mask for the word cloud


from sklearn.model_selection import train_test_split
from stop_words import get_stop_words
from collections import Counter
import string


In [3]:
# before running this make sure you have installed ibm-watson  using command  pip install ibm-watson

from ibm_watson import ToneAnalyzerV3 # importing Tone Analyzer 
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator('************************') # Enter the API key of Watson Tone Analyzer from IBM Cloud
tone_analyzer = ToneAnalyzerV3(
    version='2020-06-22', # You can enter the latest version
    authenticator=authenticator
)

tone_analyzer.set_service_url('https://api.eu-gb.tone-analyzer.watson.cloud.ibm.com/instances/1fc202da-2014-42f5-97ef-f3498040a6d3')

In [4]:
data = pd.read_csv('tone-edited.csv')# Reading a file which has all the tweets in it
df = data.copy()

In [57]:
import json

# Trying to get a csv which analyzes all the tones of the tweet with one single text

# Dictionary which conatains all the fields, text as well ad their tones 
temp_dict = {'text':[], 'anger':[], 'disgust':[], 'fear':[], 'joy':[], 'sadness':[], 'analytical':[], 'confident':[], 'tentative':[]}

# Text to be analyzed.
text = 'Team, I know that times are tough! Product '\
    'sales have been disappointing for the past three '\
    'quarters. We have a competitive product, but we '\
    'need to do a better job of selling it!'

# This gives us the result of the tone analyzer in a json format
tone_analysis = tone_analyzer.tone(
    {'text': text},
    content_type='application/json'
).get_result()

emotion_list = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'analytical', 'confident', 'tentative']

temp_dict['text'].append(text)
# The below code will add the tone analyzer score if the text contains that tone otherwise it will add 0
for i in tone_analysis['document_tone'].items():
    temp_list = []
    for j in i[1]:
        temp_dict[j['tone_id']].append(j['score'])
        temp_list.append(j['tone_id'])
    for x in emotion_list:
        if x not in temp_list:
            temp_dict[x].append(0)
                
            
dfObj = pd.DataFrame(temp_dict) 
dfObj
# json.dump(tone_analysis,open("out.json","w"))

,text,anger,disgust,fear,joy,sadness,analytical,confident,tentative
0,"Team, I know that times are tough! Product sal...",0,0,0,0,0.6165,0.829888,0,0


In [58]:
# Applying the above logic to all the tweets in the file (approx 4500 tweets)
# This will take time for me it was around 30 mins.

temp_dict = {'text':[], 'anger':[], 'disgust':[], 'fear':[], 'joy':[], 'sadness':[], 'analytical':[], 'confident':[], 'tentative':[]}

emotion_list = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'analytical', 'confident', 'tentative']


for index, review in df['tweet_text'].iteritems():
    
    tone_analysis = tone_analyzer.tone(
    {'text': review},
    content_type='application/json'
    ).get_result()
    
    temp_dict['text'].append(review)
    
    for i in tone_analysis['document_tone'].items():
        temp_list = []
        for j in i[1]:
            temp_dict[j['tone_id']].append(j['score'])
            temp_list.append(j['tone_id'])
        for x in emotion_list:
            if x not in temp_list:
                temp_dict[x].append(0)


dfObj = pd.DataFrame(temp_dict) 
dfObj

df.to_csv('processed.csv')# Storing data in  a csv for further usage.

,text,anger,disgust,fear,joy,sadness,analytical,confident,tentative
0,Please support everyone🙏🙏\n\nCorona Virus covi...,0.0,0,0.0,0.707947,0.000000,0.000000,0.00000,0.000000
1,"My heart goes out to Corona patients, who are ...",0.0,0,0.0,0.000000,0.000000,0.898696,0.00000,0.975589
2,quarantine day 10. // Good mo...,0.0,0,0.0,0.786379,0.000000,0.000000,0.00000,0.000000
3,Stay home\nStay safe\n\nLet's fight Corona tog...,0.0,0,0.0,0.579362,0.000000,0.891222,0.00000,0.000000
4,Monday Breakfast:👨‍🍳\n\nZero Stuffed Ghee Para...,0.0,0,0.0,0.840887,0.000000,0.000000,0.00000,0.000000
...,...,...,...,...,...,...,...,...,...
4564,We are still here waiting for your return when...,0.0,0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000
4565,On Sunday this is my church. I celebrate thru ...,0.0,0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000
4566,"During the PKP/MCO till PKPB/CMCO, we never ex...",0.0,0,0.0,0.562305,0.000000,0.530786,0.79901,0.000000
4567,The New Normal.. covid19 mask break.....,0.0,0,0.0,0.000000,0.520928,0.724236,0.00000,0.000000


In [101]:
df = pd.read_csv('processed.csv')
# Getting back the dataframe to do some changes.

In [102]:
# The below function will give a new column exp_name whose value will bw the column name in a particular row whose value is 
# maximum, thus which resembles the tweet sentiment the most .
df['exp_name'] = df[['anger', 'disgust', 'fear', 'joy', 'sadness', 'analytical', 'confident', 'tentative']].apply(lambda x: x.argmax(), axis=1)
df['exp_name'] = df['exp_name'].apply(lambda x: emotion_list[x])
df.head()

,Unnamed: 0,text,anger,disgust,fear,joy,sadness,analytical,confident,tentative,exp_name
0,0,Please support everyone🙏🙏\n\nCorona Virus covi...,0.0,0,0.0,0.707947,0.0,0.000000,0.0,0.000000,joy
1,1,"My heart goes out to Corona patients, who are ...",0.0,0,0.0,0.000000,0.0,0.898696,0.0,0.975589,tentative
2,2,quarantine day 10. // Good mo...,0.0,0,0.0,0.786379,0.0,0.000000,0.0,0.000000,joy
3,3,Stay home\nStay safe\n\nLet's fight Corona tog...,0.0,0,0.0,0.579362,0.0,0.891222,0.0,0.000000,analytical
4,4,Monday Breakfast:👨‍🍳\n\nZero Stuffed Ghee Para...,0.0,0,0.0,0.840887,0.0,0.000000,0.0,0.000000,joy


In [103]:
df.to_csv('processed.csv')# Storing data in  a csv for further usage.
